You can use this notebook to visulalize the quantum correlators. You need to perform a semiclassical simulation (with or without backreaction) and store "data", and "correlators". It assumes that both data-types are saved at the same timestpes. If not, you need to modify appropriately.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpecialFunctions
using FunctionZeros
using Interpolations
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
base = "../examples/quantum_runs/"

par1 = "a0.0_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"
par2 = "a0.75_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"
par3 = "a1.25_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"

dir1 = base*par1
dir2 = base*par2
dir3 = base*par3

In [ ]:
# set the number of grid points you used
D = 3
Nr = 128*2^D + 3

# make directory to save figures
out_dir = "./correlators_tr/a0.75_a1.25_comparison/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(dir1*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its1, all_filenames)           = list_h5_files(dir1*"/data_$(Nr)", prefix="data_");
(its_crlt1, all_filenames_crlt) = list_h5_files(dir1*"/data_$(Nr)", prefix="correlators_");

(its2, all_filenames)           = list_h5_files(dir2*"/data_$(Nr)", prefix="data_");
(its_crlt2, all_filenames_crlt) = list_h5_files(dir2*"/data_$(Nr)", prefix="correlators_");

(its3, all_filenames)           = list_h5_files(dir3*"/data_$(Nr)", prefix="data_");
(its_crlt3, all_filenames_crlt) = list_h5_files(dir3*"/data_$(Nr)", prefix="correlators_");

@assert its1 ≈ its2 ≈ its3
@assert its_crlt1 ≈ its_crlt2 ≈ its_crlt3

# classical data
println("length(its1) = ",length(its1))
println("its = ",its1)

# correlators
println("length(its_crlt1) = ",length(its_crlt1))
println("its_crlt = ",its_crlt1)

Define a function to plot the real part of equal time correlators. The structure of the correlators is given before the function. You can modify it to plot the imaginary part

In [ ]:
"""
for the correlators the format (r, r)
    it is the 2pt function for the fields:
    1) ∂t_utld_kl = r^l*α*πq/(B*A^0.5)
    2) ∂r_utld_kl = l*r^(l-1)*uq + r^l*ψq
    3) utld_kl = uq*r^l
    4) πq
    5) ψq
"""

function plot_crlt(dir, it, crlt_i, ri_min, ri_max)

    (its, all_filenames)           = list_h5_files(dir*"/data_$(Nr)", prefix="data_");
    (its_crlt, all_filenames_crlt) = list_h5_files(dir*"/data_$(Nr)", prefix="correlators_");
    
    # if you have not saved data and correlators on the same timesteps
    # you need to change it below, accordingly
    it_data = its[it]
    it_str_data  = lpad(it_data, 4, "0")
    
    it_crlt = its_crlt[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    
    v = h5read(dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    
    crlt = h5read(dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    
    # if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
    t_data   =  h5readattr(dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    t_crlt   =  h5readattr(dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5", "/")["time"]
    
    # checks if time instant is the same for all types of data
    @assert t_data ≈ t_crlt

    # change "real" -> "imag" below, if you want the imaginary part of the correlator
    println("max(abs(real(crlt[$(crlt_i)]))) = ",maximum(abs.(real(crlt[ri_min:ri_max, ri_min:ri_max, crlt_i]))))
    contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        # change "real" -> "imag" below, if you want the imaginary part of the correlator
        real(crlt[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r1", yaxis="r2",
        #title = L"\mathcal{R}\left[ C(r_1,r_2)\right]"*" @ t=$(t_crlt)",)
        title = "real(crlt[$(crlt_i)]) @t=$(t_crlt)",)
    AH = find_AH(r, B, A, KB)
    plot!([AH], seriestype="vline", label="AH", linewidth=3, color = "black")
    plot!([AH], seriestype="hline", label="AH", linewidth=3, color = "black")
end

In [ ]:
# index of min and max, in r domain
ri_min = 3
ri_max = 550#Nr
# timestep (counts within its_crlt list
ti = 35#69
# which correlator you want (see previous cell for the structure). There are 5 different (1-5).
crlt_i = 3

p1 = plot_crlt(dir1, ti, crlt_i, ri_min, ri_max)
p2 = plot_crlt(dir2, ti, crlt_i, ri_min, ri_max)
p3 = plot_crlt(dir3, ti, crlt_i, ri_min, ri_max)

plt = plot(
    p1, p2, p3,
    layout = grid(1, 3), size = (1200, 400))

# uncomment below if you want to save fig
#savefig(plt, out_dir*"/crlt_$(crlt_i)-$(ti).png")

In [ ]:
function load_crtl23(ti)
    it_data = its3[it]
    it_str_data  = lpad(it_data, 4, "0")
    it_crlt = its_crlt3[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    v = h5read(dir3*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    AH = find_AH(r, B, A, KB)  
    crlt3 = h5read(dir3*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    crlt2 = h5read(dir2*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    t   =  h5readattr(dir3*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    return crlt2, crlt3, t, AH
end

In [ ]:
ri_min = 3
ri_max = 345
ri = 55 # r slice
crlt_i = 3

##########################################################

it = 1
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "subcritical | t=$(t) ")

plot!([r[ri]], seriestype="vline", label="", linewidth=1.5, color = "green", legendfontsize=8)
p11 = plot!([r[ri]], seriestype="hline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "supercritical")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p12 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")

p13 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################

it = 7
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p21 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
p22 = plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
#p22 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
#plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p23 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################

it = 11
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p31 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
p32 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p33 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)


##########################################################

it = 20
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p41 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
p42 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p43 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################

it = 35
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p51 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
p52 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p53 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################

it = 50
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p61 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
p62 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p63 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################

it = 69
crlt2, crlt3, t, AH = load_crtl23(ti)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt2[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r",
        title = "t=$(t) ")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
p71 = plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)

contourf(r[ri_min:ri_max], r[ri_min:ri_max], 
        real(crlt3[ri_min:ri_max, ri_min:ri_max, crlt_i]),
        color=:thermal,
        wsize = (900,700),xaxis="r", yaxis="r")

plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)
plot!([r[ri]], seriestype="hline", label="", linewidth=1.5, color = "green", legendfontsize=8)
plot!([AH], seriestype="vline", label="", linewidth=1.5, color = "black")
p72 = plot!([AH], seriestype="hline", label="AH", linewidth=1.5, color = "black")

plot(r[ri_min:ri_max],  real(crlt2[ri_min:ri_max, ri, crlt_i]), label="subcritical", frame = true, linewidth=2)
plot!(r[ri_min:ri_max],  real(crlt3[ri_min:ri_max, ri, crlt_i]), label="supercritical", linewidth=2, style=:dash,
        xlim = (0,r[ri_max]), xlabel = "r")
    
plot!([AH], seriestype="vline", label="AH", linewidth=1.5, color = "black")
p73 = plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=1.5, color = "green", legendfontsize=8)

##########################################################
plt = plot(
    p11, p12, p13,
    p21, p22, p23,
    p31, p32, p33,
    p41, p42, p43,
    p51, p52, p53,
    p61, p62, p63,
    p71, p72, p73,
    layout = grid(7, 3), size = (1200, 7*280))

savefig(plt, out_dir*"/crlt_$(crlt_i)_snapshots_ri$(ri)_slice.pdf")

In [ ]:

"""
for the correlators the format (r, r)
    it is the 2pt function for the fields:
    1) ∂t_utld_kl = r^l*α*πq/(B*A^0.5)
    2) ∂r_utld_kl = l*r^(l-1)*uq + r^l*ψq
    3) utld_kl = uq*r^l
    4) πq
    5) ψq
"""

function plot_crlt_slice(it, crlt_i, ri_min, ri_max, ri)

    # if you have not saved data and correlators on the same timesteps
    # you need to change it below, accordingly
    it_data = its[it]
    it_str_data  = lpad(it_data, 4, "0")
    
    it_crlt = its_crlt[it]
    it_str_crlt  = lpad(it_crlt, 4, "0")
    
    v = h5read(your_dir*"/data_$(Nr)/data_$(it_str_data).h5","v")
    A  = v[:,4]
    B  = v[:,5]
    KB = v[:,9]
    
    crlt = h5read(your_dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5","correlators")
    
    # if there is an error when loading the time attribute, try to change "./" to "." or to "/" 
    t_data   =  h5readattr(your_dir*"/data_$(Nr)/data_$(it_str_data).h5", "/")["time"]
    t_crlt   =  h5readattr(your_dir*"/data_$(Nr)/correlators_$(it_str_crlt).h5", "/")["time"]
    
    # checks if time instant is the same for all types of data
    @assert t_data ≈ t_crlt
    
        plot(r[ri_min:ri_max], real(crlt[ri, ri_min:ri_max, crlt_i]),
        label="real(crlt[$(crlt_i)])",
        title = "t=$(t_crlt), r=$(r[ri])",
        linewidth = 2, frame = true, wsize = (800,400),
    xlim = (0,r[ri_max]), xlabel = "r")
    
    AH = find_AH(r, B, A, KB)
    plot!([AH], seriestype="vline", label="AH", linewidth=2, color = "black", legendfontsize=13)
    plot!([r[ri]], seriestype="vline", label="r=$(r[ri])", linewidth=2, color = "red", legendfontsize=13)
    
end

You can also plot the scalar field in time with the function below: